# Generates Mobility file for inference

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
if '..' not in sys.path:
    sys.path.append('..')
    
from matplotlib import pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import networkx as nx
import copy
import scipy as sp
import math
import seaborn
import pickle
import warnings
import os

from lib.mobilitysim_split import MobilitySimulator 
#in mobilitysim_split.py the downsample factors for population and sites are different 
from lib.town_data import generate_population, generate_sites, compute_distances
from lib.town_maps import MapIllustrator

### Settings for synthetic mobility data generation

Import __one__ `town_settings` file. The following variables will be imported by the `import *` command
* `town_name`
* `population_path`
* `sites_path`
* `bbox`
* `population_per_age_group`
* `region_population`
* `town_population`
* `daily_tests_unscaled`
* `household_info`

In [3]:
#see lib.settings folder for data from other towns 
from lib.settings.town_settings_tubingen import *

In [4]:
# Downsampling factor of population and sites 
# TO DO: introduce a different downsample factor for populations and sites
downsample_sites = 10
downsample_pop = 10

# Country for different age groups
country = 'GER' # 'GER', 'CH'

# Set the population generation mode.
# 3 options available: custom | random | heuristic
population_by = 'custom'

#### Town details

In [5]:
# Downsample population 
population_per_age_group = np.round(
    population_per_age_group * town_population / (downsample_pop * region_population)).astype('int').tolist()

print(f'Population per age group: {population_per_age_group}')

Population per age group: [534, 730, 2684, 2988, 1651, 468]


#### Extracted site data

* `site_loc`: list of site coordinates
* `site_type`: list of site category
* `site_dict`: helper dictionary with real name (string) of each site category (int)
* `density_site_loc`: list of site coordinates of specific type to be based on to generate population density

To generate sites of arbitrary sites for a given city, the following function sends queries to OpenStreetMap. In order to use it for additional types of sites, you need to specify queries in the Overpass API format. For more information, check the existing queries in **/lib/data/queries/**, https://wiki.openstreetmap.org/wiki/Overpass_API and http://overpass-turbo.eu/.

We separatelly use a query returning all buildings in a town to heuristically generate population density in the next steps if no real population density data is provided. An extra query is required for this purpose and it should be given as a **site_based_density_file** argument.

In [6]:
# This block sends queries to OpenStreetMap
# Make sure you have a working internet connection
# If an error occurs during execution, try executing again 
# If the call times out or doesn't finish, try restarting your internet connection by e.g. restarting your computer

site_files=['../lib/data/queries/education.txt','../lib/data/queries/social.txt', 
            '../lib/data/queries/bus_stop.txt','../lib/data/queries/office.txt',
            '../lib/data/queries/supermarket.txt'] 
site_loc, site_type, site_dict, density_site_loc = generate_sites(bbox=bbox, query_files=site_files,
                                site_based_density_file='../lib/data/queries/buildings.txt')

Query 1 OK.
Query 2 OK.
Query 3 OK.
Query 4 OK.
Query 5 OK.
Query 6 OK.


#### Site visualization

In [7]:
ill = MapIllustrator()
sitemap = ill.sites_map(bbox=bbox, site_loc=site_loc, site_type=site_type, site_dict = site_dict, map_name=f'{town_name}_site_distribution')
sitemap

#### Generate home location based on various options

* `home_loc`: list of home coordinates
* `people_age`: list of age category 
* `home_tile`: list of map tile to which each home belongs
* `tile_loc`: list tile center coordinates

The following three options generate a population distribution across a geographical area consisting of tiles (square boxes) of specific resolution. More information about tile sizes can be found in https://wiki.openstreetmap.org/wiki/Zoom_levels. 

In [8]:
population_path='../'+population_path

if region_population == town_population:
    tile_level = 15
else:
    tile_level = 16

if population_by == 'custom':
    # generate population across tiles based on density input
    print('Tile level: ', tile_level)
    home_loc, people_age, home_tile, tile_loc, people_household = generate_population(
        density_file=population_path, bbox=bbox,
        population_per_age_group=population_per_age_group, 
        household_info=household_info, tile_level=tile_level, seed=42)
    
elif population_by == 'random':
    # generate population across tiles uniformly at random
    home_loc, people_age, home_tile, tile_loc, people_household = generate_population(
        bbox=bbox, population_per_age_group=population_per_age_group,
        tile_level=16, seed=42)

elif population_by == 'heuristic':
    # generate population across tiles proportional to buildings per tile
    home_loc, people_age, home_tile, tile_loc, people_household = generate_population(
        bbox=bbox, density_site_loc=density_site_loc,
        population_per_age_group=population_per_age_group, tile_level=16, seed=42)

Tile level:  16


#### Home visualization

In [9]:
homemap = ill.population_map(bbox=bbox, home_loc=home_loc, map_name=f'{town_name}_population_distribution')
homemap

Downsample sites as given by settings

In [10]:
seed_downsites=42
if downsample_sites > 1:
    np.random.seed(seed_downsites) 
    # downsample sites like populatoin
    idx = np.random.choice(len(site_loc), size=int(len(site_loc) / downsample_sites), 
                           replace=False, p=np.ones(len(site_loc)) / len(site_loc))

    site_loc, site_type = np.array(site_loc)[idx].tolist(), np.array(site_type)[idx].tolist()

In [11]:
print(f'Number of sites: ', len(site_loc))
print(f'Site types:      ', site_dict)

Number of sites:  145
Site types:       {0: 'education', 1: 'social', 2: 'bus_stop', 3: 'office', 4: 'supermarket'}


Compute pairwise distances between all tile centers and all sites

In [12]:
tile_site_dist = compute_distances(site_loc, tile_loc)

In [13]:
site_type;

#### Specify synthetic mobility patterns

Here we specify the patterns of mobility used for generating the synthetic traces based on the above home and site locations. Note that this is a general framework and can by arbitrarilty extended to any desired site numbers or types. See below for an example used in the first version of our paper.

Specify the mean duration of visit per type, or in reality, time spent in crowded places per type.

In [14]:
# 2h at office-education, 1.5h at restaurants/bars, 0.5 at supermarket, 0.2 at bus stop.
dur_mean_per_type = [2, 1.5, 0.2, 2, 0.5]

Determine the number of discrete sites a person visits per site type.

In [15]:
# 1 office, 1 school, 10 social, 2 supermarkets, 5 bus stops
variety_per_type = [1, 10, 5, 1, 2]

Set the number of visits per week that each group makes per type of site

In [16]:
# e.g. line 0 corresponds to age 0-4 in Germany 
# no office, a lot of education (kindergarden), some social, no supermarket, no public transport 
# the age groups are chosen to match the age groups used in case data by national authorities

# read mobility per age per type
path_country='../lib/data/countries/'
mob_rate_read = pd.read_csv(path_country+'ger.csv',index_col='Unnamed: 0')
# convert to average visits per hour per week, to be compatible with simulator
mob_rate_per_age_per_type=np.divide(mob_rate_read.to_numpy(), (24.0 * 7))

Set `delta`; the setting for delta is explained in the paper.

In [17]:
# time horizon
#delta  = 4.6438 # as set by distributions
delta = 0.0 #zero non-contact contamination 

In [18]:
print('Population (by Age): ', population_per_age_group)
print('Sites (by type):     ',  [(np.array(site_type) == i).sum() for i in range(5)])

print('Total:', sum(population_per_age_group), len(site_type))

Population (by Age):  [534, 730, 2684, 2988, 1651, 468]
Sites (by type):      [20, 17, 37, 69, 2]
Total: 9055 145


Save arguments for the class object instantiation to be able to initiate `MobilitySimulator` on the fly during inference. That is more efficient than pickling in some cases.

In [19]:
kwargs = dict(
    home_loc=home_loc, 
    people_age=people_age, 
    site_loc=site_loc, 
    num_people_unscaled=town_population,
    region_population=region_population,
    site_type=site_type, 
    site_dict=site_dict, 
    downsample_pop=downsample_pop,
    downsample_sites=downsample_sites,
    mob_rate_per_age_per_type=mob_rate_per_age_per_type,
    daily_tests_unscaled=daily_tests_unscaled, 
    dur_mean_per_type=dur_mean_per_type, 
    variety_per_type=variety_per_type, 
    delta=delta,
    home_tile=home_tile, 
    tile_site_dist=tile_site_dist, 
    people_household=people_household)

with open(f'../lib/mobility/{town_name}_settings_pop{downsample_pop}_site{downsample_sites}.pk', 'wb') as fp:
    pickle.dump(kwargs, fp)

Create mobility traces as above, or comment in the last section below to specify fully artifial traces.

In [20]:
mob = MobilitySimulator(**kwargs)
mob.verbose = True

In [21]:
max_time = 30 * 24.0 # e.g. 17 days
%time mob.simulate(max_time=max_time,seed=12345)

Simulate mobility for 720.00 time units... Simulated 311775 visits.
Find contacts... 
CPU times: user 35.8 s, sys: 1.21 s, total: 37 s
Wall time: 36.9 s


In [22]:
%time mob.to_pickle(f'tubingen_mobility_{downsample_pop}_{downsample_sites}.pk')

CPU times: user 14.4 s, sys: 1.19 s, total: 15.6 s
Wall time: 15.6 s


#### Export continuous-time contacts
format: individual_1, individual_2, start_time, end_time, duration (in hours)

In [23]:
cont=[]
for i in mob.contacts:
    for j in mob.contacts[i]:
        for h in mob.contacts[i][j]:
            cont.append((h.indiv_i,h.indiv_j,h.t_from, h.t_to, h.t_to-h.t_from))

In [24]:
condf= pd.DataFrame(data=cont, columns=['indiv_i', 'indiv_j', 't_from', 't_to', 'deltat'])

In [25]:
condf.to_csv('contactsTubingen.csv', index=False) 

In [26]:
condf

,indiv_i,indiv_j,t_from,t_to,deltat
0,0,597,171.935842,172.105573,0.169731
1,0,2430,171.935842,172.142783,0.206941
2,0,3083,171.939103,172.142783,0.203680
3,0,3495,171.935842,172.142783,0.206941
4,0,3974,171.935842,172.029252,0.093410
...,...,...,...,...,...
3510145,9053,5140,149.224441,149.402139,0.177699
3510146,9053,5190,149.212259,149.266793,0.054534
3510147,9053,5278,149.212259,149.402139,0.189880
3510148,9053,6811,149.212259,149.402139,0.189880


In [27]:
# If necessary, load discrete-time contacts generated by the mobility model 
#!gunzip contactsTurin.csv.gz
condf=pd.read_csv("contactsTubingen.csv")
condf

,indiv_i,indiv_j,t_from,t_to,deltat
0,0,597,171.935842,172.105573,0.169731
1,0,2430,171.935842,172.142783,0.206941
2,0,3083,171.939103,172.142783,0.203680
3,0,3495,171.935842,172.142783,0.206941
4,0,3974,171.935842,172.029252,0.093410
...,...,...,...,...,...
3510145,9053,5140,149.224441,149.402139,0.177699
3510146,9053,5190,149.212259,149.266793,0.054534
3510147,9053,5278,149.212259,149.402139,0.189880
3510148,9053,6811,149.212259,149.402139,0.189880


Neglect contacts lasting less than 15 min

In [28]:
#keep resolution of 15min
contact_raw=condf[condf['deltat']>0.25]
contact_raw=contact_raw.reset_index()
contact_raw.drop(columns=['index'],inplace=True)
contact_raw

,indiv_i,indiv_j,t_from,t_to,deltat
0,0,17,294.320705,295.562099,1.241394
1,0,209,74.228038,75.344297,1.116259
2,0,387,132.438909,134.285618,1.846709
3,0,388,73.498460,75.344297,1.845838
4,0,465,294.422407,295.562099,1.139691
...,...,...,...,...,...
2318077,9053,6916,327.855158,328.580144,0.724987
2318078,9053,7047,326.623392,327.587698,0.964306
2318079,9053,7384,327.435902,330.093194,2.657292
2318080,9053,7416,326.434779,327.217648,0.782869


In [29]:
t_unit = 24
period = np.arange(0,max_time+t_unit,t_unit*1.0)
print(period)

[  0.  24.  48.  72.  96. 120. 144. 168. 192. 216. 240. 264. 288. 312.
 336. 360. 384. 408. 432. 456. 480. 504. 528. 552. 576. 600. 624. 648.
 672. 696. 720.]


In [30]:
n_contacts=len(contact_raw)
indiv_i=contact_raw.indiv_i.to_numpy()
indiv_j=contact_raw.indiv_j.to_numpy()
t_from=contact_raw.t_from.to_numpy()
t_to=contact_raw.t_to.to_numpy()
dt=contact_raw.deltat.to_numpy()    

Contacts are weighted by their duration: in case they last more than one day, they are split and counted as multiple contacts of different duration

In [31]:
cont_sqzd_dict={}
for i in range(n_contacts):
    mlen=int(dt[i]//t_unit)
    for s in range(mlen+1):
        a=(indiv_i[i],indiv_j[i],int(t_from[i])//t_unit +s)
        if a in cont_sqzd_dict:
            cont_sqzd_dict[a]+=(1 if s<mlen else (dt[i]/t_unit - mlen))
        else:
            cont_sqzd_dict[a]=(1 if s<mlen else (dt[i]/t_unit - mlen)) 

The transmission probability in a single contact $(i,j)$ is estimated as $1-e^{-\beta \Delta t_{ij}}$, where $\Delta t_{ij}$ is the duration of the continuous-time contact. The parameter $\beta$ can be straightforwardly generalized to be individual-based and time-dependent. For testing the coarse-graining approximation, values between 0.5 and 1.5 have been used; in practice it should be inferred from data.

In [32]:
beta=1.0 #between 0.5-1.5

In [33]:
cont_sqzd=[]
l=list(cont_sqzd_dict.keys())
for a in l:
    cont_sqzd+=[(a[0],a[1],a[2], 1-np.exp(-beta*cont_sqzd_dict[a]))]
    cont_sqzd+=[(a[1],a[0],a[2], 1-np.exp(-beta*cont_sqzd_dict[a]))]

In [34]:
cont2_sqzd = pd.DataFrame(cont_sqzd,columns=['i', 'j', 't', 'lambda'])

In [35]:
cont_sqzd_sorted=cont2_sqzd.sort_values(by=['t','i'],ascending=True)

In [36]:
cont_sqzd_sorted.to_csv('cont_miniTubingen.csv', index=False) 

In [37]:
cont_sqzd_sorted

,i,j,t,lambda
1954,4,753,0,0.045101
1960,4,778,0,0.056322
1962,4,1046,0,0.087391
1966,4,1186,0,0.048164
1976,4,1373,0,0.050473
...,...,...,...,...
3998775,9051,7946,29,0.046694
4550686,9051,3170,29,0.018438
4550748,9051,5307,29,0.031030
4550810,9051,7828,29,0.013198
